# [Advent of Code 2021](https://adventofcode.com/2021)

Starting from day 10, I'll try to follow [Norvig's approach](https://github.com/norvig/pytudes/blob/main/ipynb/Advent-2020.ipynb), with all solutions in a single notebook.

# Helpers

Most of the helpers are taken from Norvig's notebook.

In [1]:
import math
import re
import itertools
from collections import Counter, defaultdict, namedtuple, deque
from itertools import permutations, combinations, product
from functools import cache, reduce
from typing import Tuple

def first(iterable, default=None):
    "Return first item in `iterable`, or `default`."
    return next(iter(iterable), default)

def quantify(iterable, pred=bool) -> int:
    "Count the number of items in `iterable` for which `pred` is true."
    return sum(1 for item in iterable if pred(item))

def ints(text: str) -> Tuple[int]:
    "Return a tuple of all the integers in `text`."
    return tuple(map(int, re.findall('-?[0-9]+', text)))

def data(inp: str, parser=str, sep='\n') -> list:
    "Parse input into sections separated by `sep`, and apply `parser` to each."
    sections = inp.rstrip().split(sep)
    return [parser(section) for section in sections]

def do(day, *answers, ex=None):
    "E.g., `do(3)` returns `{1: day3_1(in3), 2: day3_2(in3)}`. Verifies `answers` if given."
    g = globals()
    got = []
    parse = g[f'parse{day}']
    data = f'ex{day}_{ex}' if ex else f'in{day}'
    for part in (1, 2):
        fname = f'day{day}_{part}'
        if fname in g: 
            got.append(g[fname](parse(g[data])))
            if len(answers) >= part:
                assert got[-1] == answers[part - 1], (
                    f'{fname}(parse{day}({data})) got {got[-1]}; expected {answers[part - 1]}')
    return got

cat = ''.join
flatten = itertools.chain.from_iterable

# Day 10: Syntax Scoring

Analyze inputs with unmatched characters.

In [2]:
ex10 = '''[({(<(())[]>[[{[]{<()<>>
[(()[<>])]({[<{<<[]>>(
{([(<{}[<>[]}>{[]{[(<()>
(((({<>}<{<{<>}{[]{[]{}
[[<[([]))<([[{}[[()]]]
[{[{({}]{}}([{[{{{}}([]
{<[[]]>}<{[{[{[]{()[[[]
[<(<(<(<{}))><([]([]()
<{([([[(<>()){}]>(<<{{
<{([{{}}[<[[[<>{}]]]>[]]'''
in10 = open('day10.txt').read()

In [3]:
def parse10(I): return data(I)
#parse10(ex10)

In [4]:
def match_errors(line):
    open, close = '([{<', ')]}>'
    s = []
    while line:
        c, *line = line
        if c in close:
            if s.pop() != open[close.index(c)]:
                return close.index(c) # unexpected character index ('corrupted' line)
        elif c in open:
            s.append(c)
    return s # unbalanced characters stack ('incomplete' line)

In [5]:
def day10_1(D):
    points = [3, 57, 1197, 25137]
    return sum(points[r] for r in map(match_errors, D) if not isinstance(r, list))
day10_1(parse10(ex10))

26397

In [6]:
day10_1(parse10(in10))

388713

In [7]:
def day10_2(D):
    def score(cs): return reduce(lambda r, c: r * 5 + '([{<'.index(c) + 1, cs, 0)
    scores = sorted(score(reversed(r)) for r in map(match_errors, D) if isinstance(r, list))
    return scores[len(scores) // 2]
day10_2(parse10(ex10))

288957

In [8]:
day10_2(parse10(in10))

3539961434

# Day 11: Dumbo Octopus

The task is to implement the algorithm that looks like [Game of Life], with notable differences:
* The grid is fixed size - 10x10. Game of Life has infinite grid.
* Calculating next step can take multiple iterations, in a [fixed-point][] fashion. In contrast, Game of Life can be implemented with a single iteration.

(See also [my solution][day17-solution] to [Advent of Code 2020, day 17][day17].)

[fixed-point]: https://en.wikipedia.org/wiki/Fixed_point_(mathematics)
[Game of Life]: https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life
[day17-solution]: https://github.com/shamrin/adventofcode/blob/main/2020/17.py
[day17]: https://adventofcode.com/2020/day/17

## Parsing

In [9]:
ex11 = '''5483143223
2745854711
5264556173
6141336146
6357385478
4167524645
2176841721
6882881134
4846848554
5283751526'''
in11 = open('day11.txt').read()

In [10]:
def parse11(I): return data(I, lambda a: list(map(int, a)))
parse11(ex11)

[[5, 4, 8, 3, 1, 4, 3, 2, 2, 3],
 [2, 7, 4, 5, 8, 5, 4, 7, 1, 1],
 [5, 2, 6, 4, 5, 5, 6, 1, 7, 3],
 [6, 1, 4, 1, 3, 3, 6, 1, 4, 6],
 [6, 3, 5, 7, 3, 8, 5, 4, 7, 8],
 [4, 1, 6, 7, 5, 2, 4, 6, 4, 5],
 [2, 1, 7, 6, 8, 4, 1, 7, 2, 1],
 [6, 8, 8, 2, 8, 8, 1, 1, 3, 4],
 [4, 8, 4, 6, 8, 4, 8, 5, 5, 4],
 [5, 2, 8, 3, 7, 5, 1, 5, 2, 6]]

## Solution core

The core of the solution is `octopus_flashes` function. It generates numbers of octopuses that flashed after each step.

Main data structure is `g` dictionary. It maps octopus *coordinate* to its current *energy value*.

Auxilary data structures:

* `step_flashes` set - coordinates of octopuses that flashed during the current step *so far*
* `flashes` set - coordinates of octopuses that flashed *right now* 
* `adds` dictionary maps *coordinate* to current flash-induced *energy increase*. For example, `adds[1,1]` tells how many octopuses just flashed next to the octopus at `(1,1)`. Uses [`collections.Counter`](https://docs.python.org/3/library/collections.html#collections.Counter) helper.

In [11]:
def octopus_flashes(I, steps):
    g = {(x,y): c for y, r in enumerate(I) for x, c in enumerate(r)}
    for _ in range(steps):
        g = {p: c+1 for p, c in g.items()}
        step_flashes = set()
        while flashes := {p for p, c in g.items() if c > 9}:
            step_flashes |= flashes
            adds = Counter((x+dx, y+dy) for x, y in flashes for dx in (-1,0,1) for dy in (-1,0,1))
            g = {p: 0 if p in step_flashes else c+adds[p] for p, c in g.items()}
        yield len(step_flashes)

## Part 1

In [12]:
def day11_1(D): return sum(octopus_flashes(D, 100))
day11_1(parse11(ex11))

1656

In [13]:
day11_1(parse11(in11))

1608

## Part 2

In [14]:
def day11_2(D): return next(i+1 for i, f in enumerate(octopus_flashes(D, 10**6)) if f == 100)
day11_2(parse11(ex11))

195

In [15]:
day11_2(parse11(in11))

214

# Day 12: Passage Pathing

## Parsing

In [16]:
ex12_1 = '''start-A
start-b
A-c
A-b
b-d
A-end
b-end'''
ex12_2 = '''dc-end
HN-start
start-kj
dc-start
dc-HN
LN-dc
HN-end
kj-sa
kj-HN
kj-dc'''
ex12_3 = '''fs-end
he-DX
fs-he
start-DX
pj-DX
end-zg
zg-sl
zg-pj
pj-he
RW-he
fs-DX
pj-RW
zg-RW
start-pj
he-WI
zg-he
pj-fs
start-RW'''
in12 = open('day12.txt').read()

In [17]:
def parse12(I):
    d = data(I, lambda s: s.split('-'))
    g = defaultdict(list)
    for v1, v2 in d:
        if v1 != 'end' and v2 != 'start':
            g[v1].append(v2)
        if v2 != 'end' and v1 != 'start':
            g[v2].append(v1)
    return g
parse12(ex12_1)

defaultdict(list,
            {'start': ['A', 'b'],
             'A': ['c', 'b', 'end'],
             'c': ['A'],
             'b': ['A', 'd', 'end'],
             'd': ['b']})

## Solution

In [18]:
def day12_1(G):
    def paths(G, start, end, path=[]):
        path = path + [start]
        if start == end: yield path
        for n in G[start]:
            if n not in path or n.isupper():
                yield from paths(G, n, end, path)
    return quantify(paths(G, 'start', 'end'))

In [19]:
def day12_2(G):
    def paths(G, start, end, path=[], twice=False):
        path = path + [start]
        if start == end: yield path
        for n in G[start]:
            if not twice or n not in path or n.isupper():
                yield from paths(G, n, end, path, twice or (not n.isupper() and n in path))
    return quantify(paths(G, 'start', 'end'))

In [20]:
do(12, 10, 36, ex=1)
do(12, 19, 103, ex=2)
do(12, 226, 3509, ex=3)
do(12, 4186, 92111)

[4186, 92111]

I found it not worth it to combine the parts together. I think they are quite readabable as they are.

# Day 13

In [ ]:
ex13_1 = ''''''
in13 = open('day13.txt').read()

In [ ]:
def parse13(I):
    return data(I, lambda s: s.split('-'))
parse13(ex13_1)

In [ ]:
def day13_1(D):
    pass

In [ ]:
def day13_2(D):
    pass

In [ ]:
do(13, ex=1)
do(13)